# Laboratorio 3

> **Tiempo:** -

> **Entrega de informe:** -


## Instalación

Para facilitar el proceso de instalación de esta actividad, trabajaremos con una máquina virtual que tendrá _casi_ todos los programas instalados.

Como motor de máquinas virtuales usaremos [Virtual Box](https://www.virtualbox.org/wiki/Downloads). Desde ese link tendrán que descargar la versión que mejor se ajuste a su sistema operativo. Luego desde los servidores de la universidad deben descargar la [máquina virtual](http://niebla.ing.puc.cl/diplomadobigdata/Lab%202.ova) ya configurada.

Finalmente debemos importar la máquina descargada dentro de Virtual Box, para ello deben seguir los siguientes pasos: Abrir virtual box > Archivo > Abrir servicio virtualizado, o bien `Crtl+I`.

**Observación:** la contraseña del usuario configurado es _ubuntu_.

Descargar este proyecto ya sea con `git` o mediante el botón de descargar y ejecutar `notebook`.

```bash
$ git clone https://github.com/stgolarrain/recsys-labs.git
$ cd recsys-labs/assignment-3
$ jupyter notebook
```

Si ya tienen el repositorio descargado en la máquina virtual, puedes actualizar el código del repositorio con el siguiente comando.

```bash
$ cd recsys-labs
$ git pull origin master
$ de assignment-3
```

Para más detalles de cómo utilizar git puedes revisar la documentación [git pull](https://git-scm.com/docs/git-pull).

Una vez descargada e importada la máquina virtual, puedes instalar las dependencias ejecutando la siguiente celda.

In [ ]:
# Instalation setup
! pip3 install nltk
! pip3 install sklearn
! pip3 install gensim
! pip3 install pandas
! pip3 install numpy

# Instrucciones

En esta oportunidad tendrán que experimentar con la librería [gensim](https://radimrehurek.com/gensim/) para el modelamiento de tópicos latentes en textos. Gensim es una librería que implementa modelos de tópicos, específicamente tendrás que trabajar con [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation). La librería también permite transformar texto no estructurado en diferentes representaciones vectoriales, tales como [TF-IDF](https://es.wikipedia.org/wiki/Tf-idf), y buscar similaridades mediante diferentes métricas de distancia.

Este laboratorio se divide en las siguientes secciones:

1. Preprocesamiento de Datos: en esta sección tendrás que descargar librerías de _Natural Language Tool Kit_, la cual implementa las funcines básicas necesarias para trabajar con texto (datos no estructurados) y transformarlos en una representación vectorial estructurada.
2. Modelo de Recomendaciones: en la segunda sección tendrás que entrenar un modelo de tópicos latentes (LDA)
3. Generar Recomendaciones: finalmente, tendrás que utilizar el modelo de tópicos para generar recomendaciones basadas en contenido para un usuario ficticio que ha consumido 3 documentos.

# 1. Preprocesamiento de Datos

In [ ]:
import os
import nltk
import sklearn
import gensim
import string
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from gensim import corpora, models, similarities
from sklearn.neighbors import NearestNeighbors

Antes de comenzar debemos descargar las librerías de lenguaje de [NLTK](https://www.nltk.org/), ejecutando la siguiente celda:

In [ ]:
# Download corpora
nltk.download()

Para comenzar cargaremos el set de datos en un _dataframe_ de _Pandas_, e imprimimos los 5 primeros registros para visualizar la estructura de los datos.

In [ ]:
corpus_df = pd.read_csv('./dataset/corpus1.csv', sep='\t', header=None, encoding='latin')
corpus_df.columns = ['id', 'title', 'abstract']
corpus_df = corpus_df[['id', 'title', 'abstract']]
corpus_df[:5]

Lo siguiente es implementar una función que transforme texto no estructurado a una lista de _tokens_ procesados.

In [ ]:
stemm = False
stemmer = PorterStemmer()

def get_tokens(text):
    lowers = text.lower()
    no_punctuation = lowers.translate(string.punctuation)
    tokens = nltk.word_tokenize(no_punctuation)
    if stemm:
        tokens = map(stemmer.stem, tokens)
    return tokens

get_tokens("I'm a super student for recommender systems!")

**Pregunta** Explique en sus palabras qué hace la función `get_tokens()`

**Respuesta**

Ahora se tiene que generar un diccionario con todas las palabras del _corpus_. Se recomiendo revisar la documentación de gensim y leer cómo usar los diccionarios. [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html)

In [ ]:
dic_file = './resources/dictionary-stemm.p' if stemm else './resources/dictionary.p'
if os.path.isfile(dic_file):
    dictionary = corpora.dictionary.Dictionary().load(dic_file)
else:
    dictionary = corpora.dictionary.Dictionary(documents=corpus_df.tokenised_abstract.tolist())
    dictionary.save(dic_file)
    
corpus_df['tokenised_abstract'] = corpus_df.abstract.map(get_tokens)
corpus_df[:5]

**Pregunta** Explique a qué corresponde la columan tokenised_abstract del dataframe

**Respuesta**

In [ ]:
corpus_df['bow'] = corpus_df.tokenised_abstract.map(dictionary.doc2bow)
del corpus_df['tokenised_abstract']
corpus = corpus_df['bow'].tolist()
corpus_df[:5]

**Pregunta** Explicar a qué corresponde la columna _bow_

**Respuesta**

# 2. Modelo de Tópicos

In [ ]:
tfidf_model_file = 'resources/tfidf_model-stemm.p' if stemm else 'resources/tfidf_model.p'
if os.path.isfile(tfidf_model_file):
    tfidf_model = models.tfidfmodel.TfidfModel().load(tfidf_model_file)
else:
    tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
    tfidf_model.save(tfidf_model_file)

# tfidf_model = models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary)
corpus_df['tf_idf'] = tfidf_model[corpus_df.bow.tolist()]
corpus_df[:5]

**Pregunta** Explicar a qué corresponde la columna tf_idf y por qué es útil en el procesamiento de texto. Mencione sus 2 principales parts, mediante la explicación del puntaje.

**Respuesta**

A continuación entrenaremos un modelo LDA.

In [ ]:
topic_number = 10

lda_model = models.LdaModel(corpus, num_topics=topic_number, id2word=dictionary, passes=5, iterations=200)
corpus_df['lda'] = lda_model[corpus_df.bow.tolist()]
corpus_df[:5]

**Pregunta** Explique qué representa la columna lda, ¿qué significan cada tupla de números?

**Respuesta**

En la siguiente celda se mostrarán 10 tópicos del modelo LDA.

In [ ]:
lda_model.print_topics(10)

**Pregunta** ¿Qué representa lo impreso en la celda anterior?

**Respuesta**

**Pregunta** A su parecer, ¿son buenos los tópicos encontrados por el modelo? ¿cómo se podrían mejorar?

**Respuesta**

# 3. Generar Recomendaciones

En esta sección se implementan las funciones necesarias para poder generar recomendaciones dado lo que un usuario ha consumido. De manera artificial, se samplearán 3 documentos aleatorios que representarán al usuario objetivo (`sample`). Luego tendrás que generar diferentes recomendaciones y evaluar los resultados.

In [ ]:
# Random user

samples = corpus_df.sample(3)

for n, (ix, paper) in enumerate(samples.iterrows()):
    idx, title, abstract, bow, tf_idf, lda = paper
    print('%d) %s' % (n+1, title))
    print('')
    print(abstract)
    print('\n' )

In [ ]:
# Recommendation functions

N = len(dictionary)

def to_sparse(matrix):
    return csr_matrix([gensim.matutils.sparse2full(row, length=N) for row in matrix]) 

def make_recommendations(model, metric, neighbors):
    M = len(corpus)

    X = to_sparse(corpus_df[model].tolist())
    document_index = NearestNeighbors(n_neighbors=(neighbors + 1), algorithm='brute', metric=metric).fit(X)
    return document_index

def print_recommendations(indexes, model):
    for n, (ix, paper) in enumerate(samples.iterrows()):
        dists, neighbors = indexes.kneighbors([gensim.matutils.sparse2full(paper[model],length=N)])
        print(paper['title'])
        print('')
        print('Documentos cercanos: ')
        i = 1
        for neighbour in neighbors[0]:
            if ix != neighbour:
                line = str(i) + ". " + corpus_df.iloc[neighbour]['title']
                print(line)
                i+=1
        print('\n')
    

A continuación deberá utilizar las funciones implementadas anteriormente para generar nuevas recomendaciones variando los parámetros del modelo. Agregue nuevas celdas para cada implementación y/o pregunta.

** Pregunta** Ejecute el modelo utilizando como representación `tf-idf` y métrica de distancia euclideana. Modifique el parámetro nearest_neighbors a [5, 10, 20]. ¿qué efecto tiene el modelo en las recomendaciones observadas?

**Respuesta**

**Pregunta** Eligiendo un valor fijo para nearest neighbors y utilizando representación `tf-idf`, ejecute el modelo con métrica de distancia _cosine_.¿Qué efecto tiene la métrica de distancia en las recomendaciones observadas?

**Respuesta**

**Pregunta** Eligiendo un valor fijo de nearest_neighbors y modelo _lda_ ¿Qué efecto tiene el usar LDA versus TF-IDF en las recomendaciones observadas?

**Respuesta**

**Pregunta** Pruebe nuevamente con LDA usando sólo 5 tópicos ¿qué efecto tiene el número de tópicos en las recomendaciones observadas?

**Respuesta**

In [ ]:
# Recommendation example
# 
# doc_idx = make_recommendations('tf_idf', 'euclidean', 5)
# print_recommendations(doc_idx, 'tf_idf')

## Entregable

Una vez completado el laboratorio y respondido las preguntas deberán exportar este archivo en formato `html` y subir a la plataforma _Moodle_.

Para exportar este archivo deben ir a `File > Donwload as > HTML (.html)`

Si tienen algún problema o duda enviar mail a **dparra [at] ing [dot] puc [dot] cl** o **slarrain [at] uc [dot] cl** anteponiendo [Diplomada Bog Data] en el asunto.